# BioSound Tutorial 1.  From wav files to h5 files.
### This notebook illustrates how to generate BioSound objects from sounds stored as wav files.  BioSound contains many parameters and it is up to the user to calculate some or all.  This script calculates all the parameters.

In the BioSound approach the sounds have been previously segmented into units (here as separate wav files) of interest. Here we are going to analyze different renditions of the distance calls of different zebra finches to examine the individual signature.  The sound units could also be different bird songs from different birds, or different speech vowel sounds or different musical instruments all playing the same note.  

## Installation instructions.

Before starting it is expected that
 1. You have downloaded all the files in github/theunissenlab/BioSoundTutorial including the sound files in github/theunissenlab/BioSoundTutorial/BirdCalls.  BirdCalls has 25 renditions of Distance Calls produced by 4 different zebra finches.
 2. You have a working version of Python and IPython to run this notebook.  The code was originally written in Python 2.7 but updated to Python 3 (3.6)
 3. You have installed the python package soundsig: <code> pip install soundsig <code>

## Step 1. Loading all the dependencies.  These should have been installed when you installed soundsig.

In [1]:
# Import math, plotting and sound libraries
import numpy as np
import matplotlib.pyplot as plt
from soundsig.sound import BioSound 
from soundsig.sound import WavFile
import os

# Make plots on this Notebook.
%matplotlib inline


## Step 2. Specifying the location of the test sound files and where to save the output.
### YOU WILL WANT TO CHANGE TO FOLLOWING CODE TO POINT TO THE RIGHT PLACE



In [2]:
# Go to the folder that has the wav files
#os.chdir('/Users/mathevon/Documents/ANALYSE_SONS/BIOSOUND/BioSoundTutorial-master/BirdCalls')

#os.chdir('/Users/mathevon/Documents/ANALYSE_SONS/BIOSOUND/PleurSansSilence')
os.chdir('/Users/frederictheunissen/Google Drive/My Drive/PowerData/PleursBebe/cry_sequences_22babies')

# This will be the output directory
if not os.path.exists('h5files'):
    os.makedirs('h5files')
    


## Step3. Loop through all the files in the folder, make BioSound objects and plot the first two.

The plotting routine in BioSound (.plot()) generates a variable number of graphs depending on what has been calculated.  The first figure shows the oscillogram and spectrogram of the sound.  The red bold line on the oscillogram is the amplitude enveloppe.  The black bold line on the spectrogram is the estimated fundamental frequency. The purple bold line on the spectrogram is the estimated fundamental frequency of a second voice (when it exists). The red, white and blue bold lines on the specrogram are the estimated first, second and third formants (F1, F2, F3).  The second figure is a power spectrum. The black dotted lines show the quartiles.  The red, light blue and dark blue dotted lines are the time average values of F1,F2, F3. The third figure shows the values of other BioSound parameters.  The fourth figure shows the modulation power spectrum.
The calculations last approximately 1 minute for a 1 second sound on 2.7 GHz Intel Core i7.  The slowest routine is the fundamental estimator. 

In [3]:
# Plot flag to decide whether or not to plot output
plotMe = True
normalize = False

# Find all the wave files 
isound = 0   
babynames = []
crytypes = []
filenames = os.listdir('.')
for fname in filenames :
    if fname.endswith('.wav'):
        isound += 1;
        
        # Read the sound file
        #print ('Found sound %d:%s\n' % (isound, fname))
        # soundIn = WavFile(file_name=fdir+'/'+fname) 
        filename, file_extension = os.path.splitext(fname)
        
        
        # Here we parse the filename to get the birdname and the call type. 
        # You will have to write custom code to extract your own identifiers.
        crytype = filename[4]    # <Condition : bain ou vqccin plus pays
        babyname = filename[0:3]  
            
        print ('Condition', crytype, 'Baby ID', babyname)
        babynames.append(babyname)
        crytypes.append(crytype)

print('Found %d babies' % len(np.unique(babynames)))

Condition D Baby ID ASE
Condition D Baby ID FAE
Condition B Baby ID FAE
Condition B Baby ID RAR
Condition B Baby ID ASE
Condition B Baby ID CAA
Condition B Baby ID CAA
Condition B Baby ID ASE
Condition B Baby ID RAR
Condition B Baby ID FAE
Condition D Baby ID RAR
Condition D Baby ID FAE
Condition D Baby ID CAA
Condition B Baby ID VIC
Condition D Baby ID FAE
Condition B Baby ID CAE
Condition B Baby ID ASE
Condition B Baby ID CAA
Condition B Baby ID RAR
Condition B Baby ID FAE
Condition B Baby ID VIC
Condition B Baby ID BEV
Condition D Baby ID ASE
Condition D Baby ID CAA
Condition D Baby ID FAE
Condition D Baby ID RAR
Condition B Baby ID FAE
Condition B Baby ID RAR
Condition B Baby ID ASE
Condition B Baby ID CAA
Condition B Baby ID CAA
Condition B Baby ID CAA
Condition B Baby ID CAA
Condition B Baby ID ASE
Condition B Baby ID RAR
Condition B Baby ID FAE
Condition D Baby ID RAR
Condition D Baby ID CAA
Condition D Baby ID ASE
Condition B Baby ID BEV
Condition D Baby ID ASE
Condition B Baby

In [4]:
# Plot flag to decide whether or not to plot output
plotMe = True
normalize = True
import wave

# Find all the wave files 
isound = 0 
filenames = os.listdir('.')
for fname in filenames :
    if fname.endswith('.wav'):
        isound += 1
        
        # Read the sound file
                
        print ('Processing sound %d:%s\n' % (isound, fname))
        soundIn = WavFile(file_name=fname) 
        
        filename, file_extension = os.path.splitext(fname)
        
        
        # Here we parse the filename to get the birdname and the call type. 
        # You will have to write custom code to extract your own identifiers.
        crytype = filename[4]    # <Condition : bain ou vqccin plus pays
        babyname = filename[0:3]  

  
        print ('Condition', crytype, 'ID', babyname)     
        # Normalize if wanted
        if normalize :
            maxAmp = np.abs(soundIn.data).max() 
        else :
            maxAmp = 1.0
    
    # Create BioSound Object
        myBioSound = BioSound(soundWave=soundIn.data.astype(float)/maxAmp, fs=float(soundIn.sample_rate), emitter=babyname, calltype = crytype)
             
    # Calculate the spectrogram and the rms
        myBioSound.spectroCalc(spec_sample_rate=1000, freq_spacing = 50, min_freq=0, max_freq=10000)
        myBioSound.rms = myBioSound.sound.std() 
       
    # Calculate amplitude enveloppe
        myBioSound.ampenv(cutoff_freq = 20, amp_sample_rate = 1000)
       
    # Calculate the power spectrum
        myBioSound.spectrum(f_high=10000)
    
    # Calculate fundamental and related values.  These are the default values.
    # For the estimation of the fundamental, four methods are available: 
    # 'AC' - Peak of the auto-correlation function
    # 'ACA' - Peak of envelope of auto-correlation function 
    # 'Cep' - First peak in cepstrum 
    # 'Stack' - Fitting of harmonic stacks (default - works well for zebra finches)
    
        myBioSound.fundest(maxFund = 1500, minFund = 300, lowFc = 200, highFc = 6000, minSaliency = 0.5, debugFig = 0, minFormantFreq = 500, maxFormantBW = 500, method='Stack')
      
    # Calculate the MPS
        myBioSound.mpsCalc(window=1, Norm = True)
                
        if plotMe and isound <= 5: 
            print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
            print('                Baby %s    Condition %s' % (myBioSound.emitter, myBioSound.type))
            myBioSound.plot(DBNOISE=50, f_low=250, f_high=10000)  
            print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    

    # Save the results
        fh5name = 'h5files/%s.h5' % (filename)
        myBioSound.saveh5(fh5name)


Processing sound 1:ASE_D04.wav

